# Recipes For Translating A BUGS Model To LogDensityProblems

# `Rats` Model

## JuliaBUGS Model Definition
```julia
@bugsast begin
    for i in 1:N
        for j in 1:T
            Y[i, j] ~ dnorm(mu[i, j], tau_c)
            mu[i, j] = alpha[i] + beta[i] * (x[j] - xbar)
        end
        alpha[i] ~ dnorm(alpha_c, alpha_tau)
        beta[i] ~ dnorm(beta_c, beta_tau)
    end
    tau_c ~ dgamma(0.001, 0.001)
    sigma = 1 / sqrt(tau_c)
    alpha_c ~ dnorm(0.0, 1.0E-6)
    alpha_tau ~ dgamma(0.001, 0.001)
    beta_c ~ dnorm(0.0, 1.0E-6)
    beta_tau ~ dgamma(0.001, 0.001)
    alpha0 = alpha_c - xbar * beta_c
end
```

In [1]:
using LogDensityProblems
using UnPack
using Distributions
using ReverseDiff
using JuliaBUGS.BUGSPrimitives

In [20]:
# data
x = [8.0, 15.0, 22.0, 29.0, 36.0]
xbar = 22
N = 30
T = 5
Y = [
    151 199 246 283 320
    145 199 249 293 354
    147 214 263 312 328
    155 200 237 272 297
    135 188 230 280 323
    159 210 252 298 331
    141 189 231 275 305
    159 201 248 297 338
    177 236 285 350 376
    134 182 220 260 296
    160 208 261 313 352
    143 188 220 273 314
    154 200 244 289 325
    171 221 270 326 358
    163 216 242 281 312
    160 207 248 288 324
    142 187 234 280 316
    156 203 243 283 317
    157 212 259 307 336
    152 203 246 286 321
    154 205 253 298 334
    139 190 225 267 302
    146 191 229 272 302
    157 211 250 285 323
    132 185 237 286 331
    160 207 257 303 345
    169 216 261 295 333
    157 205 248 289 316
    137 180 219 258 291
    153 200 244 286 324
]

# parameters
alpha = ones(Integer, 30) .* 250
beta = ones(Integer, 30) .* 6
alpha_c = 150
beta_c = 10
tau_c = 1
alpha_tau = 1
beta_tau = 1

alpha = ones(Integer, 30) .* 25
beta = ones(Integer, 30) .* 0.6
alpha_c = 15
beta_c = 1
tau_c = 0.1
alpha_tau = 0.1
beta_tau = 0.1

initializations = Dict(
    :alpha => alpha,
    :beta => beta,
    :alpha_c => alpha_c,
    :beta_c => beta_c,
    :tau_c => tau_c,
    :alpha_tau => alpha_tau,
    :beta_tau => beta_tau,
);

In [12]:
# `LogDensityProblems` type definition: put data here
struct Rats
    x
    xbar::Float64
    Y
    N::Int
    T::Int
end

# With [AdvancedHMC.jl](https://github.com/TuringLang/AdvancedHMC.jl)

HMC needs parameters to be unconfined, the second argument of function `dnorm` is required to be positive. We can use the package [`Bijectors.jl`](https://github.com/TuringLang/Bijectors.jl).

In [17]:
using Bijectors
b = Bijectors.Log{0}()
b⁻¹ = Bijectors.Exp{0}();

In [21]:
# parameters should live in unconstrained space, do the transformation first
tau_c = b(initializations[:tau_c]) # 1 => 0
alpha_tau = b(initializations[:alpha_tau]) # 1 => 0
beta_tau = b(initializations[:beta_tau]); # 1 => 0

In [8]:
function (problem::Rats)(parameters)
    @unpack x, xbar, Y, N, T = problem
    alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau = parameters[1:N], parameters[N+1:2N], parameters[2N+1:end]... # parameters are linearized

    b⁻¹ = Bijectors.Exp{0}()
    tau_c = b⁻¹(tau_c)
    alpha_tau = b⁻¹(alpha_tau)
    beta_tau = b⁻¹(beta_tau)
    
    loglikelihood = 0.0
    for i in 1:N
        for j in 1:T
            mu_i_j = alpha[i] + beta[i] * (x[j] - xbar)
            loglikelihood += logpdf(dnorm(mu_i_j, tau_c), Y[i, j])
        end
    end
    for i in 1:N
        loglikelihood += logpdf(dnorm(alpha_c, alpha_tau), alpha[i])
        loglikelihood += logpdf(dnorm(beta_c, beta_tau), beta[i])
    end

    loglikelihood += logpdf(dgamma(0.001, 0.001), tau_c)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), alpha_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), alpha_tau)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), beta_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), beta_tau)
    return loglikelihood
end

function LogDensityProblems.logdensity(p::Rats, parameters)
    return p(parameters)
end

function LogDensityProblems.capabilities(::Type{<:Rats})
    return LogDensityProblems.LogDensityOrder{1}()
end

function LogDensityProblems.dimension(p::Rats)
    @unpack x, xbar, Y, N, T = p
    return N + N + 1 + 1 + 1 + 1 + 1
end

In [22]:
p = Rats(x, xbar, Y, N, T)
initial_θ = convert(Array{Float64}, vcat(alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau))
p(initial_θ)

-380769.7837397596

In [10]:
# check gradient
using ReverseDiff
g = ReverseDiff.gradient(p, initial_θ)

65-element Vector{Float64}:
    -151.0
    -110.0
     -86.0
    -189.0
    -194.0
    -100.0
    -209.0
    -107.0
      74.0
    -258.0
     -56.0
    -212.0
    -138.0
       ⋮
     -94.0
     557.0
     326.0
     -87.0
    -122.0
    -234.0
      60.0
  -23486.0
    2999.99985
 -149986.0
    -120.00001
    -226.0

In [11]:
alpha_g, beta_g, tau_c_g, alpha_c_g, alpha_tau_g, beta_c_g, beta_tau_g = g[1:N], g[N+1:2N], g[2N+1:end]...

([-151.0, -110.0, -86.0, -189.0, -194.0, -100.0, -209.0, -107.0, 74.0, -258.0  …  -106.0, -227.0, -210.0, -124.0, -179.0, -78.0, -76.0, -135.0, -265.0, -143.0], [18.0, 648.0, 284.0, -444.0, 340.0, 88.0, -38.0, 242.0, 648.0, -122.0  …  235.0, -115.0, -185.0, -94.0, 557.0, 326.0, -87.0, -122.0, -234.0, 60.0], -23486.0, 2999.99985, -149986.0, -120.00001, -226.0)

In [14]:
for v in [:alpha, :beta, :tau_c, :alpha_c, :alpha_tau, :beta_c, :beta_tau]
    println(v)
    println("values ", eval(v))
    println("gradients ", eval(Symbol(String(v) * "_g")))
    println()
end

alpha
values [250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250]
gradients [-151.0, -110.0, -86.0, -189.0, -194.0, -100.0, -209.0, -107.0, 74.0, -258.0, -56.0, -212.0, -138.0, -4.0, -136.0, -123.0, -191.0, -148.0, -79.0, -142.0, -106.0, -227.0, -210.0, -124.0, -179.0, -78.0, -76.0, -135.0, -265.0, -143.0]

beta
values [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
gradients [18.0, 648.0, 284.0, -444.0, 340.0, 88.0, -38.0, 242.0, 648.0, -122.0, 487.0, 53.0, 81.0, 417.0, -395.0, -73.0, 151.0, -122.0, 235.0, 11.0, 235.0, -115.0, -185.0, -94.0, 557.0, 326.0, -87.0, -122.0, -234.0, 60.0]

tau_c
values 0.0
gradients -23486.0

alpha_c
values 150
gradients 2999.99985

alpha_tau
values 0.0
gradients -149986.0

beta_c
values 10
gradients -120.00001

beta_tau
values 0.0
gradients -226.0



In [15]:
using AdvancedHMC

D = length(initial_θ)
n_samples, n_adapts = 2000, 1000

metric = DiagEuclideanMetric(D)
hamiltonian = Hamiltonian(metric, p, :ReverseDiff)

initial_ϵ = find_good_stepsize(hamiltonian, initial_θ)
integrator = Leapfrog(initial_ϵ)
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; drop_warmup=true, progress=true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Sampling 100%|███████████████████████████████| Time: 0:01:21
  iterations:                                   2000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.8144367646130171
  log_density:                                  -650.0488705312952
  hamiltonian_energy:                           680.6510388645884
  hamiltonian_energy_error:                     0.1403567251441018
  max_hamiltonia

([[241.69908470264178, 247.88702895924078, 253.66032071185717, 231.0357089901332, 236.44109170375435, 251.30586134173163, 225.04939557102654, 249.67409710407486, 283.45753312490234, 211.6175886943941  …  6.604353229014514, 5.827784160157115, 5.914671778427448, 5.625789106818496, 5.924997115195007, -3.5942226726390176, 243.11073238751052, -5.4941120722017684, 6.196442474835487, 1.565629432368299], [238.7210369715854, 247.91445110525834, 251.5488041267646, 233.29624829221606, 228.29497885533797, 248.44294248569602, 231.255384731698, 244.3357895831155, 285.8287040286001, 225.12971773019035  …  6.137138931180131, 5.816276981881468, 5.6958506142220555, 5.655790557339817, 6.201081397511731, -3.6679633437694164, 243.7580663653776, -5.583759930635426, 6.214108131830969, 1.0118147858845066], [240.70541782239417, 248.14321526119457, 253.10357446404217, 232.38787889466383, 234.7936740538887, 251.38666840943307, 226.7729190961257, 252.46487787847008, 280.79158028663, 214.4382458673218  …  6.887574

In [16]:
beta_c_samples = [samples[s][64] for s in 1:length(samples)]
stats = mean(beta_c_samples), std(beta_c_samples) # Reference result: mean 6.186, variance 0.1088

(6.1944702500081315, 0.1123019774522192)

In [17]:
tau_c_samples = [samples[s][61] for s in 1:length(samples)]
sigma_samples = map(x->1/sqrt(b⁻¹(x)), tau_c_samples)
stats = mean(sigma_samples), std(sigma_samples) # Reference result: mean 6.092, sd 0.4672

(6.1243127618231235, 0.45942245753857064)

# With [DynamicHMC.jl](https://github.com/tpapp/DynamicHMC.jl)

In [5]:
function (problem::Rats)(parameters)
    @unpack x, xbar, Y, N, T = problem
    @unpack alpha, beta, tau_c, alpha_c, alpha_tau, beta_c, beta_tau = parameters
    
    loglikelihood = 0.0
    for i in 1:N
        for j in 1:T
            mu_i_j = alpha[i] + beta[i] * (x[j] - xbar) # array mutation is tricky, assign to a intermediate value
            loglikelihood += logpdf(dnorm(mu_i_j, tau_c), Y[i, j])
        end
    end
    for i in 1:N
        loglikelihood += logpdf(dnorm(alpha_c, alpha_tau), alpha[i])
        loglikelihood += logpdf(dnorm(beta_c, beta_tau), beta[i])
    end

    loglikelihood += logpdf(dgamma(0.001, 0.001), tau_c)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), alpha_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), alpha_tau)
    loglikelihood += logpdf(dnorm(0.0, 1.0E-6), beta_c)
    loglikelihood += logpdf(dgamma(0.001, 0.001), beta_tau)
    return loglikelihood
end

function LogDensityProblems.logdensity(p::Rats, parameters)
    return p(parameters)
end

function LogDensityProblems.capabilities(::Type{<:Rats})
    return LogDensityProblems.LogDensityOrder{1}()
end

function LogDensityProblems.dimension(p::Rats)
    @unpack x, xbar, Y, N, T = p
    return N + N + 1 + 1 + 1 + 1 + 1
end

In [6]:
p = Rats(x, xbar, Y, N, T)
p((alpha = alpha, beta = beta, tau_c = tau_c, alpha_c = alpha_c, alpha_tau = alpha_tau, beta_c = beta_c, beta_tau = beta_tau))

-174029.38703951906

In [7]:
using TransformVariables, TransformedLogDensities
t = as((alpha = as(Array, N), beta = as(Array, N), tau_c = asℝ₊, alpha_c = asℝ, alpha_tau = asℝ₊, beta_c = asℝ, beta_tau = asℝ₊))
t_p = TransformedLogDensity(t, p)

TransformedLogDensity of dimension 65

In [8]:
using LogDensityProblemsAD, DynamicHMC, Random, ReverseDiff
∇P = ADgradient(:ReverseDiff, t_p)

ReverseDiff AD wrapper for TransformedLogDensity of dimension 65 (no compiled tape)

In [9]:
inits = (alpha = alpha, beta = beta, tau_c = tau_c, alpha_c = alpha_c, alpha_tau = alpha_tau, beta_c = beta_c, beta_tau = beta_tau)
results = mcmc_with_warmup(Random.GLOBAL_RNG, ∇P, 1000)

┌ Info: found initial stepsize
└   ϵ = 0.000644
┌ Info: Starting MCMC
│   total_steps = 75
└   tuning = "stepsize"
┌ Info: MCMC progress
│   step = 1
│   seconds_per_step = 0.00034
│   estimated_seconds_left = 0.025
└   ϵ = 0.000644
┌ Info: Starting MCMC
│   total_steps = 25
└   tuning = "stepsize and LinearAlgebra.Diagonal metric"
┌ Info: MCMC progress
│   step = 1
│   seconds_per_step = 0.053
│   estimated_seconds_left = 1.3
└   ϵ = 0.0837
┌ Info: adaptation finished
└   adapted_kinetic_energy = Gaussian kinetic energy (Diagonal), √diag(M⁻¹): [1.3023899740882627, 1.77893453697697, 2.227594348026749, 3.0585138889223544, 3.223032690400119, 2.928755984679208, 3.616863804308044, 2.440392264215118, 2.024844639044452, 2.702983397294678, 2.492961156471274, 2.7213134934701686, 2.7544816686155102, 3.608846024236104, 1.583213494510248, 2.2105135558621303, 1.5814656922333028, 2.3027306861597316, 2.407048476971131, 2.3295079859520014, 2.7081506201212533, 2.037596600365919, 3.314698073314701, 1.0

(posterior_matrix = [239.16736575397027 241.7092598032288 … 238.2792733214518 241.46839714371438; 248.7863674414097 251.68958835068656 … 242.81601352826175 252.11604258612698; … ; 6.274318598875769 6.378869008280547 … 6.160539691083838 6.211124371437655; 1.803314069077694 1.045205378301353 … 2.060169705641662 1.563414753325403], tree_statistics = DynamicHMC.TreeStatisticsNUTS[DynamicHMC.TreeStatisticsNUTS(-688.536982818734, 3, turning at positions -6:1, 0.8567878658413584, 7, DynamicHMC.Directions(0x22485ec1)), DynamicHMC.TreeStatisticsNUTS(-688.047539447077, 3, turning at positions -6:1, 0.7584446227793616, 7, DynamicHMC.Directions(0x2f377e79)), DynamicHMC.TreeStatisticsNUTS(-682.4031378477603, 3, turning at positions -6:1, 0.9432550867882135, 7, DynamicHMC.Directions(0x02f307a9)), DynamicHMC.TreeStatisticsNUTS(-685.4571039394845, 3, turning at positions -5:2, 0.9769022097153529, 7, DynamicHMC.Directions(0x023198aa)), DynamicHMC.TreeStatisticsNUTS(-686.1855205318818, 3, turning at pos

In [10]:
# check the posterior of variable `beta_c`
transformed_results = transform.(t, eachcol(results.posterior_matrix))
beta_c_samples = [transformed_results[s][:beta_c] for s in 1:length(transformed_results)]
stats = mean(beta_c_samples), std(beta_c_samples) # reference results: mean 6.186, variance 0.1088

(6.187927942084446, 0.10527951971888205)